# SNNベースAIチャットシステム Colabクイックスタート

このノートブックは、SNNプロジェクトの主要な機能をGoogle Colab上で簡単に実行するためのガイドです。

**主な内容:**
1.  **環境設定**: プロジェクトのセットアップと必要なライブラリのインストール
2.  **自律エージェントの実行**: 学習済みモデルを使った推論と、オンデマンドでの新規モデル学習
3.  **高次認知プランナーの操作**: 複数の専門家を連携させた複雑なタスクの解決
4.  **モデルの学習**: `train.py` を使ったSNNモデルの学習プロセス


## 1. 環境設定

まず、プロジェクトのセットアップスクリプトを実行して、必要なライブラリをインストールします。
（注：実際の環境では、まず `git clone` でリポジトリをクローンする必要があります）

In [ ]:
!bash setup_colab.sh

## 2. 自律エージェント (`run_agent.py`) の実行

`AutonomousAgent` にタスクを依頼します。エージェントは自己の能力（モデル登録簿）を調べ、最適な専門家モデルを選択するか、必要であれば新しい専門家をオンデマンドで学習します。

### 例1: 新しい専門家モデル（文章要約）をオンデマンドで学習

まだ「文章要約」の専門家はいないため、エージェントは提供されたデータ (`data/sample_data.jsonl`) を使って自動的に知識蒸留を開始します。

In [ ]:
!python run_agent.py \
    --task_description "文章要約" \
    --unlabeled_data_path data/sample_data.jsonl \
    --prompt "SNNは、生物の神経系における情報の伝達と処理のメカニズムを模倣したニューラルネットワークの一種である。"

### 例2: 既存の専門家モデル（感情分析）を選択して推論

（このセルを実行する前に、「感情分析」モデルが学習済みであると仮定します。初回は学習が走ります）

In [ ]:
!python run_agent.py \
    --task_description "感情分析" \
    --unlabeled_data_path data/sample_data.jsonl \
    --prompt "この映画は本当に素晴らしかった！"

## 3. 高次認知プランナー (`run_planner.py`) の操作

プランナーは、複雑なタスクを複数のサブタスクに分解し、段階的に解決します。

### ステップ 3.1: 知識ベースの構築

プランナーが内部のRAG（Retrieval-Augmented Generation）システムで利用する知識ベースを構築します。

In [ ]:
!python scripts/build_knowledge_base.py

### ステップ 3.2: 複雑なタスクの実行

プランナーに「要約」と「感情分析」を組み合わせたタスクを依頼します。プランナーは動的に計画を立て、対応する専門家を順番に呼び出します。

In [ ]:
!python run_planner.py \
    --task_request "この記事を要約して、その内容の感情を分析してください。" \
    --context_data "SNNは非常にエネルギー効率が高いことで知られているが、その性能はまだANNに及ばない点もある。"

## 4. モデルの学習 (`train.py`)

知識蒸留を用いて、小型の専門家SNNを学習させる例です。

まず、蒸留のための教師データ（ロジット）を事前計算します。

In [ ]:
!python scripts/prepare_distillation_data.py \
    --input_file data/sample_data.jsonl \
    --output_dir precomputed_data/sample_distill

次に、事前計算したデータを使ってSNNモデルを学習します。

In [ ]:
!python train.py \
    --config configs/base_config.yaml \
    --model_config configs/models/small.yaml \
    --data_path precomputed_data/sample_distill \
    --override_config "training.type=distillation"